# 6) Compare parameter space

This analyzes the distance in the parameter space across different feature strategies.

In [ ]:
import pickle
import pandas as pd
import seaborn as sns
import sys
import shutil

import bluepyopt
import bluepyopt as bpopt
import bluepyopt.ephys as ephys

import matplotlib.pyplot as plt
from scipy.spatial import distance
import MEAutility as mu
import json
import time
import numpy as np
from pathlib import Path

from scipy.stats import kruskal, mannwhitneyu, wilcoxon

import multimodalfitting as mf

%matplotlib notebook

## Load GT params and optimization output

In [ ]:
base_dir = Path("../..")

In [ ]:
# general
model_name = "hay_ais"
probe_type = "planar" # linear 

cell_models_folder = base_dir / "cell_models"
model_folder = cell_models_folder / model_name

In [ ]:

result_dates = [r.name for r in (base_dir / "results").iterdir()]
# use latest results
results_date = max(result_dates)
result_folder = base_dir / "results" / results_date

In [ ]:
cell = mf.create_ground_truth_model(model_name=model_name,
                                    release=False)
cell_release = mf.create_ground_truth_model(model_name=model_name,
                                            release=True)

probe = mf.define_electrode(probe_type=probe_type)

param_names = [param.name for param in cell.params.values() if not param.frozen]
# sim = ephys.simulators.LFPySimulator(cell, cvode_active=True, electrode=probe, mechs_folders=model_folder)

params_release = {}
for param in cell_release.params_by_names(param_names):
    params_release[param.name] = param.value

In [ ]:
protocol_for_eap = "IDrest_300"

In [ ]:
pkl_file_name = "runs.pkl"

In [ ]:
data = pickle.load(open(result_folder / pkl_file_name, 'rb'))
df_optimization = pd.DataFrame(data)
df_model = df_optimization.query(f"model == '{model_name}'")

with open(result_folder / f"all_responses_{model_name}.pkl", "rb") as f:
    opt_results_training =  pickle.load(f)

In [ ]:
# save fig
save_fig = True
fig_folder = Path("../figures")

if save_fig:
    fig_folder.mkdir(exist_ok=True)

## Compare parameters 

Here we normalize the parameters based on the boundaries and compute the relative difference to GT.

In [ ]:
param_json = model_folder / "parameters.json"

with param_json.open() as f:
    params = json.load(f)

param_boundaries = {}
for param in params:
    if "bounds" in param:
        if isinstance(param['sectionlist'], list):
            for sec in param['sectionlist']:
                param_boundaries[f"{param['param_name']}_{sec}"] = param["bounds"]
        else:
            sec = param['sectionlist']
            param_boundaries[f"{param['param_name']}_{sec}"] = param["bounds"]

# scale params_release by boundaries
params_release_norm = {}
for param_name, param_val in params_release.items():
    bounds = param_boundaries[param_name]
    param_norm = (param_val - bounds[0]) / (bounds[1] - bounds[0])
    params_release_norm[param_name] = param_norm

In [ ]:
feature_set_array = []
seed_array = []
param_name_array = []
param_value_array = []
param_norm_array = []
release_value_array = []
release_norm_array = []
diff_with_release_array = []
section_array = []


# HERE we gather data from all seeds
for i, (index, opt) in enumerate(df_model.iterrows()):
    params_dict = {k: v for k, v in zip(param_names, opt.best_params)}
    for param_name, param_value in params_dict.items():
        feature_set_array.append(opt.strategy)
        seed_array.append(opt.seed)   
        param_value_array.append(param_value)
        if "axon_initial_segment" in param_name:
            section_array.append("ais")
        else:
            section_array.append(param_name.split("_")[-1])
        release_value_array.append(params_release[param_name])
        release_norm_array.append(params_release_norm[param_name])
        # compute norm value
        bounds = param_boundaries[param_name]
        param_norm = (param_value - bounds[0]) / (bounds[1] - bounds[0])
        param_norm_array.append(param_norm)
        diff_with_release_array.append(abs(param_norm - params_release_norm[param_name]))
        if "axon_initial_segment" in param_name:
            param_name = param_name.replace("axon_initial_segment", "ais")
        param_name_array.append(param_name)

for param_name, param_value in params_release.items():
    feature_set_array.append("GT")
    seed_array.append(0)
    param_value_array.append(param_value)
    section_array.append(param_name.split("_")[-1])
    release_value_array.append(params_release[param_name])
    release_norm_array.append(params_release_norm[param_name])
    # compute norm value
    bounds = param_boundaries[param_name]
    param_norm = (param_value - bounds[0]) / (bounds[1] - bounds[0])
    param_norm_array.append(param_norm)
    diff_with_release_array.append(0)
    if "axon_initial_segment" in param_name:
        param_name = param_name.replace("axon_initial_segment", "ais")
    param_name_array.append(param_name)

df_params = pd.DataFrame({"seed": seed_array, "feature_set": feature_set_array, 
                          "param_name": param_name_array,
                          "param_value": param_value_array, "param_norm": param_norm_array, 
                          "release_value": release_value_array, "release_norm": release_norm_array,
                          "diff_release": diff_with_release_array, "section": section_array})

In [ ]:
df_no_gt = df_params.query("feature_set != 'GT'")

In [ ]:
# overall parameter diff
fig_all, ax_all = plt.subplots(figsize=(10, 7))
sns.boxenplot(data=df_no_gt, y="feature_set", x="param_norm", ax=ax_all)
ax_all.set_xlabel("Normalized parameter distance", fontsize=15)
ax_all.set_ylabel("Strategy", fontsize=15)
ax_all.spines["top"].set_visible(False)
ax_all.spines["right"].set_visible(False)

In [ ]:
if save_fig:
    fig_all.savefig(fig_folder / "figS3-all.pdf")

In [ ]:
section = "somatic"
fig_split, ax_split = plt.subplots(figsize=(10, 7))
sns.barplot(data=df_no_gt.query(f"section == '{section}'"), x="param_name", y="param_norm", hue="feature_set",
            orientation="vertical", ax=ax_split)
ax_split.set_ylabel("Normalized parameter distance", fontsize=15)
ax_split.set_xlabel("Parameter name", fontsize=15)
new_labels = [l.get_text()[:l.get_text().find(f"_{section}")] for l in ax_split.get_xticklabels()]
ax_split.set_xticklabels(new_labels, fontsize=10, rotation=45)
fig_split.subplots_adjust(bottom=0.3)
ax_split.set_title(section, fontsize=20)
ax_split.spines["top"].set_visible(False)
ax_split.spines["right"].set_visible(False)

fig_sec, ax_sec = plt.subplots(figsize=(10, 7))
sns.boxenplot(data=df_no_gt.query(f"section == '{section}'"), y="feature_set", x="param_norm", ax=ax_sec)
ax_sec.set_xlabel("Normalized parameter distance", fontsize=15)
ax_sec.set_ylabel("Strategy", fontsize=15)
ax_sec.set_title(section, fontsize=20)
ax_sec.spines["top"].set_visible(False)
ax_sec.spines["right"].set_visible(False)

In [ ]:
if save_fig:
    fig_split.savefig(fig_folder / "figS3-soma-params.pdf")
    fig_sec.savefig(fig_folder / "figS3-soma-all.pdf")

In [ ]:
section = "apical"
fig_split, ax_split = plt.subplots(figsize=(10, 7))
sns.barplot(data=df_no_gt.query(f"section == '{section}'"), x="param_name", y="param_norm", hue="feature_set",
            orientation="vertical", ax=ax_split)
ax_split.set_ylabel("Normalized parameter distance", fontsize=15)
ax_split.set_xlabel("Parameter name", fontsize=15)
new_labels = [l.get_text()[:l.get_text().find(f"_{section}")] for l in ax_split.get_xticklabels()]
ax_split.set_xticklabels(new_labels, fontsize=10, rotation=45)
fig_split.subplots_adjust(bottom=0.3)
ax_split.set_title(section, fontsize=20)
ax_split.spines["top"].set_visible(False)
ax_split.spines["right"].set_visible(False)

fig_sec, ax_sec = plt.subplots(figsize=(10, 7))
sns.boxenplot(data=df_no_gt.query(f"section == '{section}'"), y="feature_set", x="param_norm", ax=ax_sec)
ax_sec.set_xlabel("Normalized parameter distance", fontsize=15)
ax_sec.set_ylabel("Feature set", fontsize=15)
ax_sec.set_title(section, fontsize=20)
ax_sec.spines["top"].set_visible(False)
ax_sec.spines["right"].set_visible(False)

In [ ]:
if save_fig:
    fig_split.savefig(fig_folder / "figS3-apical-params.pdf")
    fig_sec.savefig(fig_folder / "figS3-apical-all.pdf")

In [ ]:
section = "basal"
fig_split, ax_split = plt.subplots(figsize=(10, 7))
sns.barplot(data=df_no_gt.query(f"section == '{section}'"), x="param_name", y="param_norm", hue="feature_set",
            orientation="vertical", ax=ax_split)
ax_split.set_ylabel("Normalized parameter distance", fontsize=15)
ax_split.set_xlabel("Parameter name", fontsize=15)
new_labels = [l.get_text()[:l.get_text().find(f"_{section}")] for l in ax_split.get_xticklabels()]
ax_split.set_xticklabels(new_labels, fontsize=10, rotation=45)
fig_split.subplots_adjust(bottom=0.3)
ax_split.set_title(section, fontsize=20)
ax_split.spines["top"].set_visible(False)
ax_split.spines["right"].set_visible(False)

fig_sec, ax_sec = plt.subplots(figsize=(10, 7))
sns.boxenplot(data=df_no_gt.query(f"section == '{section}'"), y="feature_set", x="param_norm", ax=ax_sec)
ax_sec.set_xlabel("Normalized parameter distance", fontsize=15)
ax_sec.set_ylabel("Feature set", fontsize=15)
ax_sec.set_title(section, fontsize=20)
ax_sec.spines["top"].set_visible(False)
ax_sec.spines["right"].set_visible(False)

In [ ]:
if save_fig:
    fig_split.savefig(fig_folder / "figS3-basal-params.pdf")
    fig_sec.savefig(fig_folder / "figS3-basal-all.pdf")

In [ ]:
section = "ais"
fig_split, ax_split = plt.subplots(figsize=(10, 7))
sns.barplot(data=df_no_gt.query(f"section == '{section}'"), x="param_name", y="param_norm", hue="feature_set",
            orientation="vertical", ax=ax_split)
ax_split.set_ylabel("Normalized parameter distance", fontsize=15)
ax_split.set_xlabel("Parameter name", fontsize=15)
new_labels = [l.get_text()[:l.get_text().find(f"_{section}")] for l in ax_split.get_xticklabels()]
ax_split.set_xticklabels(new_labels, fontsize=10, rotation=45)
fig_split.subplots_adjust(bottom=0.3)
ax_split.set_title(section, fontsize=20)
ax_split.spines["top"].set_visible(False)
ax_split.spines["right"].set_visible(False)

fig_sec, ax_sec = plt.subplots(figsize=(10, 7))
sns.boxenplot(data=df_no_gt.query(f"section == '{section}'"), y="feature_set", x="param_norm", ax=ax_sec)
ax_sec.set_xlabel("Normalized parameter distance", fontsize=15)
ax_sec.set_ylabel("Feature set", fontsize=15)
ax_sec.set_title(section, fontsize=20)
ax_sec.spines["top"].set_visible(False)
ax_sec.spines["right"].set_visible(False)

In [ ]:
if save_fig:
    fig_split.savefig(fig_folder / "figS3-ais-params.pdf")
    fig_sec.savefig(fig_folder / "figS3-ais-all.pdf")